# E-Commerce Multi-File Data Cleaning & Consolidation
##### This project simulates a real-world scenario where data is spread across multiple inconsistent CSV files.
##### The goal is to clean, validate, standardize, and merge the datasets into one final analysis-ready master file.
#### Files Used:
##### - customers.csv
##### - orders.csv
##### - products.csv
##### - payments.csv
#### Final Deliverables:
##### - Cleaned individual files
##### - A consolidated master dataset
##### - Before vs after documentation
##### - Exported .csv output ready for reporting and analytics

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## Step 1: Load All Raw Files
##### First, load the raw CSV files to inspect structure, missing values, and inconsistencies.

In [12]:
customers=pd.read_csv("customers.csv")
products=pd.read_csv("products.csv")
orders=pd.read_csv("orders.csv")
payments=pd.read_csv("payments.csv")
customers.shape , products.shape , orders.shape , payments.shape

((7, 5), (6, 4), (7, 5), (6, 5))

In [13]:
customers.head() , products.head() , orders.head() , payments.head()

(  customer_id      customer_name                    email       city segment
 0        C001        Arjun Singh    arjun.singh@gmail.com     Mumbai     B2C
 1        C002        Riya Sharma                      NaN      Delhi     B2C
 2        C003  Tech Corp Pvt Ltd      contact@techcorp.in  Bangalore     B2B
 3        C004        Mohan Patel  mohan.patel@example.com  Ahmedabad     NaN
 4        C002        Riya Sharma    riya.sharma@gmail.com      Delhi     B2C,
   product_id    product_name     category  unit_price
 0       P001  Wireless Mouse  Electronics       899.0
 1       P002    Laptop Stand  Electronics      1499.0
 2       P003    Office Chair    Furniture      5999.0
 3       P004       Desk Lamp  Electronics         NaN
 4       P005    Notebook Set   Stationery       299.0,
   order_id customer_id product_id  order_date  quantity
 0    O1001        C001       P001  2024/01/15         2
 1    O1002        C002       P003  01-17-2024         1
 2    O1003        C004      

## Step 2: Data Audit for Each File
##### Here I inspect each raw dataframe using .info() to understand:
##### - Row counts and column counts  
##### - Data types (object, int, float, datetime)  
##### - How many non-null values exist in each column  
##### This helps me see which files are messier and which fields need the most cleaning.

In [15]:
print("Customers info:")
print(customers.info(), "\n")
print("Orders info:")
print(orders.info(), "\n")
print("Products info:")
print(products.info(), "\n")
print("Payments info:")
print(payments.info(), "\n")

Customers info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    6 non-null      object
 1   customer_name  7 non-null      object
 2   email          5 non-null      object
 3   city           7 non-null      object
 4   segment        5 non-null      object
dtypes: object(5)
memory usage: 412.0+ bytes
None 

Orders info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_id     7 non-null      object
 1   customer_id  6 non-null      object
 2   product_id   7 non-null      object
 3   order_date   7 non-null      object
 4   quantity     7 non-null      int64 
dtypes: int64(1), object(4)
memory usage: 412.0+ bytes
None 

Products info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 en

## Step 3: Cleaning customers Data
##### Goals:
##### - Ensure each customer_id is unique.
##### - Remove obviously invalid rows.
##### - Handle missing segment and email in a sensible way.
##### - Standardize text fields for consistency.

In [17]:
customers_clean = customers.copy()
# Strip spaces from string columns
for col in ["customer_id", "customer_name", "email", "city", "segment"]:
    customers_clean[col] = customers_clean[col].astype(str).str.strip()
# Replace "nan" strings back to real NaN
customers_clean = customers_clean.replace({"nan": np.nan, "None": np.nan, "": np.nan})
# Drop rows where customer_id is missing – cannot use these in joins
customers_clean = customers_clean.dropna(subset=["customer_id"])
# Drop exact duplicate rows
customers_clean = customers_clean.drop_duplicates()
# If same customer_id appears multiple times, keep the one with the most info (non-null fields)
customers_clean = customers_clean.sort_values( by=["customer_id", "email"], na_position="last").drop_duplicates(subset=["customer_id"], keep="first")
# Fill missing segment based on simple rule: B2B if company-like, else B2C
mask_b2b = customers_clean["customer_name"].str.contains("Ltd|Enterprises|Corp", case=False, na=False)
customers_clean.loc[mask_b2b & customers_clean["segment"].isna(), "segment"] = "B2B"
customers_clean["segment"] = customers_clean["segment"].fillna("B2C")
# If email missing, fill with placeholder (for client work we usually keep row but mark it)
customers_clean["email"] = customers_clean["email"].fillna("unknown@example.com")
customers_clean.reset_index(drop=True, inplace=True)
customers_clean

,customer_id,customer_name,email,city,segment
0,C001,Arjun Singh,arjun.singh@gmail.com,Mumbai,B2C
1,C002,Riya Sharma,riya.sharma@gmail.com,Delhi,B2C
2,C003,Tech Corp Pvt Ltd,contact@techcorp.in,Bangalore,B2B
3,C004,Mohan Patel,mohan.patel@example.com,Ahmedabad,B2C
4,C005,Sai Enterprises,support@sai-enterprises.com,Hyderabad,B2B


## Step 4: Cleaning products Data
##### Goals:
##### - Fix pricing values and types.
##### - Handle missing unit_price.
##### - Remove duplicate product rows.
##### - Keep a clean product catalog for joins.

In [19]:
products_clean = products.copy()
# Strip spaces
for col in ["product_id", "product_name", "category"]: products_clean[col] = products_clean[col].astype(str).str.strip()
# Fix data type of unit_price
products_clean["unit_price"] = pd.to_numeric(products_clean["unit_price"], errors="coerce")
# Handle missing prices: fill with median per category (simple, explainable rule)
products_clean["unit_price"] = products_clean.groupby("category")["unit_price"].transform(lambda x: x.fillna(x.median()))
# If still missing (all NaN in category), fill with global median
products_clean["unit_price"] = products_clean["unit_price"].fillna(products_clean["unit_price"].median())
# Drop exact duplicate rows
products_clean = products_clean.drop_duplicates()
# Ensure unique product_id – keep first occurrence if duplicated
products_clean = products_clean.sort_values("product_id").drop_duplicates(subset=["product_id"], keep="first")
products_clean.reset_index(drop=True, inplace=True)
products_clean

,product_id,product_name,category,unit_price
0,P001,Wireless Mouse,Electronics,899.0
1,P002,Laptop Stand,Electronics,1499.0
2,P003,Office Chair,Furniture,5999.0
3,P004,Desk Lamp,Electronics,1199.0
4,P005,Notebook Set,Stationery,299.0


## Step 5: Cleaning orders Data
##### Goals:
##### - Standardize order_date into a proper datetime format.
##### - Remove duplicate orders.
##### - Handle missing or invalid customer_id and quantity.
##### - Prepare a clean orders table suitable for joining with customers and products.

In [21]:
orders_clean = orders.copy()
# Strip spaces
for col in ["order_id", "customer_id", "product_id", "order_date"]: orders_clean[col] = orders_clean[col].astype(str).str.strip()
# Convert quantity to numeric and drop non-positive or invalid quantities
orders_clean["quantity"] = pd.to_numeric(orders_clean["quantity"], errors="coerce")
orders_clean = orders_clean[orders_clean["quantity"] > 0]
# Parse dates – let pandas infer formats
orders_clean["order_date"] = pd.to_datetime(orders_clean["order_date"], errors="coerce", infer_datetime_format=True)
# Drop rows where order_date could not be parsed
orders_clean = orders_clean.dropna(subset=["order_date"])
# Drop rows where customer_id is missing – cannot link them
orders_clean = orders_clean.dropna(subset=["customer_id"])
# Drop exact duplicates (same order + product + quantity)
orders_clean = orders_clean.drop_duplicates(subset=["order_id", "product_id", "quantity"])
orders_clean.reset_index(drop=True, inplace=True)
orders_clean

,order_id,customer_id,product_id,order_date,quantity
0,O1001,C001,P001,2024-01-15,2
1,O1006,C005,P003,2024-01-25,2


## Step 6: Cleaning payments Data
##### Goals:
##### - Keep only one payment per order.
##### - Focus on successful payments for the final master dataset.
##### - Fix missing amount where possible, or drop unusable rows.

In [23]:
payments_clean = payments.copy()
for col in ["payment_id", "order_id", "payment_method", "payment_status"]:payments_clean[col] = payments_clean[col].astype(str).str.strip()
# Convert amount to numeric
payments_clean["amount"] = pd.to_numeric(payments_clean["amount"], errors="coerce")
# Drop exact duplicate rows
payments_clean = payments_clean.drop_duplicates()
# For final sales view, we only care about successful payments
payments_success = payments_clean[payments_clean["payment_status"].str.lower() == "success"].copy()
# If amount is missing for a successful payment, we will try to recompute later after joining with orders/products.
payments_success.reset_index(drop=True, inplace=True)
payments_clean, payments_success

(  payment_id order_id payment_method payment_status   amount
 0       PM01    O1001            UPI        Success   1798.0
 1       PM02    O1002    Credit Card         Failed   5999.0
 2       PM03    O1003            UPI        Success   4497.0
 4       PM04    O1005           Cash        Pending      NaN
 5       PM05    O1006     NetBanking        Success  11998.0,
   payment_id order_id payment_method payment_status   amount
 0       PM01    O1001            UPI        Success   1798.0
 1       PM03    O1003            UPI        Success   4497.0
 2       PM05    O1006     NetBanking        Success  11998.0)

## Quick sanity check on cleaned tables to see how many rows survived

In [25]:
print("Customers (raw vs clean):", customers.shape, "→", customers_clean.shape)
print("Products (raw vs clean):", products.shape, "→", products_clean.shape)
print("Orders (raw vs clean):", orders.shape, "→", orders_clean.shape)
print("Payments (raw vs clean):", payments.shape, "→", payments_clean.shape)
print("Successful payments:", payments_success.shape)

Customers (raw vs clean): (7, 5) → (5, 5)
Products (raw vs clean): (6, 4) → (5, 4)
Orders (raw vs clean): (7, 5) → (2, 5)
Payments (raw vs clean): (6, 5) → (5, 5)
Successful payments: (3, 5)


## Step 7: Merge Cleaned Tables into a Single Master Dataset
##### Now that each dataset has been cleaned individually, we will merge them using shared keys:
##### - `orders_clean` joins with `customers_clean` using `customer_id`
##### - `orders_clean` joins with `products_clean` using `product_id`
##### - `orders_clean` joins with `payments_success` using `order_id`
##### This will create one final analysis-ready dataset.

In [27]:
# Start with orders as the base
master = orders_clean.merge(customers_clean, on="customer_id", how="left")
# Add product information
master = master.merge(products_clean, on="product_id", how="left")
# Add successful payment information
master = master.merge(payments_success[["order_id", "payment_method", "payment_status", "amount"]], on="order_id", how="left")
master

,order_id,customer_id,product_id,order_date,quantity,customer_name,email,city,segment,product_name,category,unit_price,payment_method,payment_status,amount
0,O1001,C001,P001,2024-01-15,2,Arjun Singh,arjun.singh@gmail.com,Mumbai,B2C,Wireless Mouse,Electronics,899.0,UPI,Success,1798.0
1,O1006,C005,P003,2024-01-25,2,Sai Enterprises,support@sai-enterprises.com,Hyderabad,B2B,Office Chair,Furniture,5999.0,NetBanking,Success,11998.0


## Step 8: Feature Engineering
##### We add meaningful business columns such as:
##### - expected_amount → calculated using quantity × unit_price
##### - payment_difference → difference between expected and actual paid amount
##### - order_month and order_year for time-based reporting

In [50]:
master["expected_amount"] = master["quantity"] * master["unit_price"]
# Difference between expected and paid amount
master["payment_difference"] = master["expected_amount"] - master["amount"]
# Extract date components
master["order_month"] = master["order_date"].dt.month
master["order_year"] = master["order_date"].dt.year
master

,order_id,customer_id,product_id,order_date,quantity,customer_name,email,city,segment,product_name,category,unit_price,payment_method,payment_status,amount,expected_amount,payment_difference,order_month,order_year
0,O1001,C001,P001,2024-01-15,2,Arjun Singh,arjun.singh@gmail.com,Mumbai,B2C,Wireless Mouse,Electronics,899.0,UPI,Success,1798.0,1798.0,0.0,1,2024
1,O1006,C005,P003,2024-01-25,2,Sai Enterprises,support@sai-enterprises.com,Hyderabad,B2B,Office Chair,Furniture,5999.0,NetBanking,Success,11998.0,11998.0,0.0,1,2024


## Step 9: Export Clean Files
##### We save the final cleaned versions of individual files and the consolidated master dataset to the `data_clean/` folder.

In [31]:
customers_clean.to_csv("customers_clean.csv", index=False)
products_clean.to_csv("products_clean.csv", index=False)
orders_clean.to_csv("orders_clean.csv", index=False)
payments_success.to_csv("payments_success.csv", index=False)
master.to_csv("master_dataset.csv", index=False)

## Step 10: Summary Output Table

In [33]:
master.isna().sum()

order_id              0
customer_id           0
product_id            0
order_date            0
quantity              0
customer_name         0
email                 0
city                  0
segment               0
product_name          0
category              0
unit_price            0
payment_method        0
payment_status        0
amount                0
expected_amount       0
payment_difference    0
order_month           0
order_year            0
dtype: int64

In [34]:
master.describe()

,order_date,quantity,unit_price,amount,expected_amount,payment_difference,order_month,order_year
count,2,2.0,2.000000,2.000000,2.000000,2.0,2.0,2.0
mean,2024-01-20 00:00:00,2.0,3449.000000,6898.000000,6898.000000,0.0,1.0,2024.0
min,2024-01-15 00:00:00,2.0,899.000000,1798.000000,1798.000000,0.0,1.0,2024.0
25%,2024-01-17 12:00:00,2.0,2174.000000,4348.000000,4348.000000,0.0,1.0,2024.0
50%,2024-01-20 00:00:00,2.0,3449.000000,6898.000000,6898.000000,0.0,1.0,2024.0
75%,2024-01-22 12:00:00,2.0,4724.000000,9448.000000,9448.000000,0.0,1.0,2024.0
max,2024-01-25 00:00:00,2.0,5999.000000,11998.000000,11998.000000,0.0,1.0,2024.0
std,NaN,0.0,3606.244584,7212.489168,7212.489168,0.0,0.0,0.0
